In [1]:
from typing_extensions import TypedDict,List
from langgraph.graph.state import StateGraph, START,END
from langgraph.graph.message import add_messages

In [23]:
from typing import Annotated
from numpy import add


class SubgraphState(TypedDict):
    a: str  
    b: str  
    f: str
    arr: Annotated[list,add_messages]

def subgraph_node_1(state: SubgraphState):
    return {"b": "biswas",}

def subgraph_node_2(state: SubgraphState):
    return {"f": state["a"]+ state['b'] }

def add_in_arr(state: SubgraphState):
    return {"arr": 'arr'}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_node(add_in_arr)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph_builder.add_edge("subgraph_node_2", "add_in_arr")
subgraph = subgraph_builder.compile()

class ParentState(TypedDict):
    a: str
    b: str
    f: str
    arr: Annotated[list,add_messages]

def node_1(state: ParentState):
    return {"a": "hi! " + state["a"]}

builder = StateGraph(ParentState)
builder.add_node( node_1)
builder.add_node("node_2", subgraph)
builder.add_node("node_3",add_in_arr)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)
graph = builder.compile()

In [24]:
for chunk in graph.stream({"a": "anik"}):
    print(chunk)

{'node_1': {'a': 'hi! anik'}}
{'node_2': {'a': 'hi! anik', 'b': 'biswas', 'f': 'hi! anikbiswas', 'arr': [HumanMessage(content='arr', additional_kwargs={}, response_metadata={}, id='e447a4e1-7f69-4739-8f02-965e142e1ea9')]}}
{'node_3': {'arr': 'arr'}}


In [25]:
graph.invoke({"a": "anik"})

{'a': 'hi! anik',
 'b': 'biswas',
 'f': 'hi! anikbiswas',
 'arr': [HumanMessage(content='arr', additional_kwargs={}, response_metadata={}, id='8fab32df-fbd1-481e-bf58-9c2da10328fc'),
  HumanMessage(content='arr', additional_kwargs={}, response_metadata={}, id='e00692bb-e7bd-49d5-b881-bcd9eedc2a9c')]}